鸭子类型（duck typing）：只需要按照预定行为实现对象所需的方法即可

Python 提供了两种获取对象的字符串表示形式的方式
- repr()
    - 便于开发者理解的方式
- str()
    - 便于用户理解的方式

给对象提供其他的表示形式的特殊方法
- \_\_bytes__
    - bytes() 函数调用它获取对象的字节序列表示形式
- \_\_format__
    - format() 函数和 str.format() 方法调用，使用特殊的格式代码显示对象的字符串表示形式


In [1]:
from array import array
import math


class Vector2d:
    # 类属性，Vector2d 实例和字节序列之间转换时使用
    typecode = 'd'
    
    # 转换成浮点数
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    # 可迭代，拆包
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    # 使用 {!r} 获取各个分量的表示形式，然后插值构成字符串
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    # 得到元组
    def __str__(self):
        return str(tuple(self))

    # 字节序列
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # 转换成字节序列
                bytes(array(self.typecode, self)))  # 数组转换成字节序列
    
    # 比较
    # [x] 两个实例都是 Vector2d
    # Vector(3, 4) == [3, 4]
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    # 直角三角形的斜边长
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    # 模
    def __bool__(self):
        return bool(abs(self))
    
    # 类方法
    @classmethod
    def frombytes(cls, octets):  # 通过 cls 传入类本身
        typecode = chr(octets[0])  # 从第一个字节中读取 typecode
        memv = memoryview(octets[1:]).cast(typecode)  # 从字节序列创建 memoryview ，然后使用 typecode 转换
        return cls(*memv)  # 拆包转换后的 memoryview ，得到构造方法所需的一对参数

In [2]:
v1 = Vector2d(3, 4)
v1

In [3]:
# 通过属性访问
v1.x, v1.y

In [4]:
# 拆包
x, y = v1
x, y

In [5]:
# repr 函数调用 Vector2d 实例得到的是对构造方法的准确描述
v1_clone = eval(repr(v1))

In [6]:
# 支持 == 比较
v1 == v1_clone

In [7]:
# 调用 __bytes__ 方法
octets = bytes(v1)
octets

In [8]:
# 调用 __abs__ 方法
abs(v1)

5.0

In [9]:
# 调用 __bool__ 方法
bool(v1), bool(Vector2d(0, 0))

(True, False)

classmethod
- 定义操作类的方法，改变了调用方法的方式，因此类方法的第一个参数是类本身
- 最常见的用途时定义备选构造方法
- 按照约定，类方法的第一个参数名为 cls

staticmethod
- 改变方法的调用方式
- 静态方法就是普通的函数，只是在类的定义体中，而不是在模块层定义

In [10]:
class Demo:
    @classmethod
    def classmeth(*args):
        return args  # 返回全部位置参数
    
    @staticmethod
    def staticmeth(*args):
        return args  # 返回全部位置参数

In [11]:
# 第一个参数始终是 Demo 类
Demo.classmeth()

(__main__.Demo,)

In [12]:
Demo.classmeth('spam')

(__main__.Demo, 'spam')

In [13]:
# 和普通函数的行为相似
Demo.staticmeth()

()

In [14]:
Demo.staticmeth('spam')

('spam',)

内置的 forma() 函数和 str.format() 方法把各个类型的格式化方式委托给相应的 .\_\_format__(format_spec) 方法。

format_spec 是格式说明符
- format(my_obj, format_spec) 的第二个参数
- str.format() 方法的格式字符串，{} 里代换字段中冒号后面的部分

In [15]:
brl = 1/2.43
brl

0.4115226337448559

In [16]:
# 保留小数点后两位
format(brl, '0.4f')

'0.4115'

In [17]:
# rate 子串是字段名称
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.41 USD'

`{0.mass:5.3e}`
- 0.mass 是字段名
- 5.3e 是格式说明符
    - 格式说明符使用的表示法叫做格式规范微语言（Format Specification Mini-Language）

格式规范微语言
- 为一些内置类型提供了专用的表示代码
- 是可扩展的，因为各个类可以自行确定如何解释 format_spec 参数

In [18]:
# b 二进制
# x 十六进制
format(52, 'b')

'110100'

In [19]:
# f 小数形式
# % 百分数形式
format(2/3, '.1%')

'66.7%'

In [20]:
# datetime 模块中给的类，其 __format__ 方法使用的格式代码与 strftime() 函数一样
from datetime import datetime

now = datetime.now()
format(now, '%H:%M:%S')

'19:49:54'

In [21]:
"It's now {:%I:%M %p}".format(now)

"It's now 07:49 PM"

如果没有定义 \_\_format__ 方法，从 object 继承的方法会返回 str(my_object)；如果传入格式说明符，object.\_\_format__ 会抛出 TypeError

In [22]:
# 定义了 __str__ 方法
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [23]:
# 没有定义 __format__ 方法
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [24]:
from array import array
import math


class Vector2d:
    # 类属性，Vector2d 实例和字节序列之间转换时使用
    typecode = 'd'
    
    # 转换成浮点数
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    # 可迭代，拆包
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    # 使用 {!r} 获取各个分量的表示形式，然后插值构成字符串
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    # 得到元组
    def __str__(self):
        return str(tuple(self))

    # 字节序列
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # 转换成字节序列
                bytes(array(self.typecode, self)))  # 数组转换成字节序列
    
    # 比较
    # [x] 两个实例都是 Vector2d
    # Vector(3, 4) == [3, 4]
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    # 直角三角形的斜边长
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    # 模
    def __bool__(self):
        return bool(abs(self))
    
    # 类方法
    @classmethod
    def frombytes(cls, octets):  # 通过 cls 传入类本身
        typecode = chr(octets[0])  # 从第一个字节中读取 typecode
        memv = memoryview(octets[1:]).cast(typecode)  # 从字节序列创建 memoryview ，然后使用 typecode 转换
        return cls(*memv)  # 拆包转换后的 memoryview ，得到构造方法所需的一对参数
    
    # 格式化字符串
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)

In [25]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [26]:
format(v1, '.2f')

'(3.00, 4.00)'

In [27]:
from array import array
import math


class Vector2d:
    # 类属性，Vector2d 实例和字节序列之间转换时使用
    typecode = 'd'
    
    # 转换成浮点数
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    # 可迭代，拆包
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    # 使用 {!r} 获取各个分量的表示形式，然后插值构成字符串
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    # 得到元组
    def __str__(self):
        return str(tuple(self))

    # 字节序列
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # 转换成字节序列
                bytes(array(self.typecode, self)))  # 数组转换成字节序列
    
    # 比较
    # [x] 两个实例都是 Vector2d
    # Vector(3, 4) == [3, 4]
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    # 直角三角形的斜边长
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    # 模
    def __bool__(self):
        return bool(abs(self))
    
    # 类方法
    @classmethod
    def frombytes(cls, octets):  # 通过 cls 传入类本身
        typecode = chr(octets[0])  # 从第一个字节中读取 typecode
        memv = memoryview(octets[1:]).cast(typecode)  # 从字节序列创建 memoryview ，然后使用 typecode 转换
        return cls(*memv)  # 拆包转换后的 memoryview ，得到构造方法所需的一对参数
    
    
    # 计算角度
    def angle(self):
        return math.atan2(self.x, self.y)
    
    # 格式化字符串
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):  # 自定义格式代码，显示极坐标
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

In [28]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [29]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [30]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

In [31]:
from array import array
import math


class Vector2d:
    # 类属性，Vector2d 实例和字节序列之间转换时使用
    typecode = 'd'
    
    # 私有
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
    
    # 只读属性
    @property
    def x(self):
        return self.__x
    
    # 只读属性
    @property
    def y(self):
        return self.__y
    
    # 可散列
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    # 可迭代，拆包
    def __iter__(self):
        return (i for i in (self.x, self.y))  # 读取公开属性
    
    # 使用 {!r} 获取各个分量的表示形式，然后插值构成字符串
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    # 得到元组
    def __str__(self):
        return str(tuple(self))

    # 字节序列
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # 转换成字节序列
                bytes(array(self.typecode, self)))  # 数组转换成字节序列
    
    # 比较
    # [x] 两个实例都是 Vector2d
    # Vector(3, 4) == [3, 4]
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    # 直角三角形的斜边长
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    # 模
    def __bool__(self):
        return bool(abs(self))
    
    # 类方法
    @classmethod
    def frombytes(cls, octets):  # 通过 cls 传入类本身
        typecode = chr(octets[0])  # 从第一个字节中读取 typecode
        memv = memoryview(octets[1:]).cast(typecode)  # 从字节序列创建 memoryview ，然后使用 typecode 转换
        return cls(*memv)  # 拆包转换后的 memoryview ，得到构造方法所需的一对参数
    
    
    # 计算角度
    def angle(self):
        return math.atan2(self.x, self.y)
    
    # 格式化字符串
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):  # 自定义格式代码，显示极坐标
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    

In [32]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)

hash(v1), hash(v2)

(7, 384307168202284039)

In [33]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

名称改写（name mangling）：以 \_\_mood （两个前导下划线，尾部没有或最多有一个下划线）的形式命名实例属性，Python 会把属性名存入实例的 \_\_dict__ 属性中，而且会在前面加上一个下划线和类名

名称改写是一种安全措施，目的是避免意外访问

Python 解释器不会对单个下划线的属性名做特殊处理，不过这是很多 Python 程序员严格遵守的约定，它们不会再类外部访问这种属性
- 不过在模块中，顶层名称使用一个前导下划线的话，的确会有影响
    - 对 from mymod import * 来说，mymod 中前缀为下划线的名称不会被导入
    - 不过仍然可以使用 from mymod import \_privatefunc 导入
    
Python 文档有时将单个下划线前缀标记的属性称为“受保护”的属性，也有人称之为“私有”属性

In [34]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [35]:
v1._Vector2d__x

3.0

默认情况下，Python 在各个实例中名为 \_\_dict__ 的字典里存储实例属性。通过 \_\_slots__ 类属性，让解释器在元组中存储实例属性，而不是用 \_\_dict__ 字典。

继承自超类的 \_\_slots__ 属性没有效果，Python 只会使用个各类中定义的 \_\_slots__ 属性。

定义 \_\_slots__ 的方式是，创建一个类属性，使用 \_\_slots__ 这个名字，并把它的值设置为一个字符串构成的可迭代对象，其中各个元素表示各个实例属性。

在类中定义 \_\_slots__ 属性之后，实例不能再有 \_\_slots__ 中所列名称之外的其他属性。

\_\_slots__ 是用于优化的。

In [36]:
class Vector2d:
    __slots__ = ('__x', '__y')

    typecode = 'd'
    
    ...

\_\_weakref__ 属性，为了让对象支持弱引用，必须有这个属性。用户定义的类中默认就有 \_\_weakref__ 属性。

如果类中定义了 \_\_slots__ 属性，而且想把实例作为弱引用的目标，那么就要把 \_\_weakref__ 添加到 \_\_slots__ 中。


使用 \_\_slots__ 的注意事项
- 每个子类都要定义 \_\_slots__ 属性
- 实例只能拥有 \_\_slots__ 中列出的属性
- 如果不把 \_\_weakref__ 加入 \_\_slots__ ，实例就不能作为弱引用的目标

In [37]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    # 没有硬编码 class_name 便于继承
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

Python 中的类属性可用于为实例提供默认值，如果为不存在的实例属性赋值，会新建实例属性

In [38]:
v1 = Vector2d(1.1, 2.2)

dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [39]:
len(dumpd)

17

In [40]:
# 修改实例属性
v1.typecode = 'f'

dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [41]:
len(dumpf)

9

In [42]:
Vector2d.typecode

'd'

In [43]:
# 修改类属性
Vector2d.typecode = 'f'

In [44]:
# 覆盖 typecode 类属性
class ShortVector2d(Vector2d):
    typecode = 'f'

sv = ShortVector2d(1/11, 1/27)
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [45]:
len(bytes(sv))

9

Python 之禅：简洁胜于复杂。

符合 Python 风格的对象应该正好符合所需，而不是堆砌语言特性。

要构建符合 Python 风格的对象，就要观察真正的 Python 对象的行为。

\_\_index__ 方法的作用是强制把对象转换成整数索引，在特定的切片场景中使用，以及满足 NumPy 的一个需求。